# utils

In [ ]:
#|default_exp utils

In [ ]:
#|hide
from nbdev.showdoc import show_doc

In [ ]:
#|export
from pathlib import Path
from typing import Union
import os

from nblite.config import NBLiteConfig, get_project_root_and_config, CodeLocation, read_config
from nblite.const import file_exts_to_format, nblite_config_file_name

In [ ]:
import nblite.utils

In [ ]:
show_doc(nblite.utils.get_nb_format_from_path)

---

### get_nb_format_from_path

>      get_nb_format_from_path (path:str)

In [ ]:
#|export
def get_nb_format_from_path(path: str) -> str:
    path = Path(path).as_posix()
    for ext, fmt in file_exts_to_format.items():
        if path.endswith(f".{ext}"): return fmt
    return None

In [ ]:
get_nb_format_from_path('file.pct.py')

'module'

In [ ]:
show_doc(nblite.utils.get_nb_path_info)

---

### get_nb_path_info

>      get_nb_path_info (nb_path:str, root_path:str,
>                        config:nblite.config.NBLiteConfig)

In [ ]:
#|export
def get_nb_path_info(nb_path: str, root_path: str, config: NBLiteConfig):
    nb_path = Path(nb_path).resolve()
    root_path = Path(root_path).resolve()
    rel_nb_path = nb_path.relative_to(root_path)
    if not rel_nb_path.parts[0] in [loc.path for loc in config.code_locations.values()]:
        raise ValueError(f"Notebook '{nb_path}' is not in a valid code location.")
        
    for loc in config.code_locations.values():
        if str(rel_nb_path).startswith(loc.path):
            file_ext = loc.file_ext
            if not str(rel_nb_path).endswith(file_ext):
                raise ValueError(f"Notebook '{nb_path}' has an invalid file extension.")
            name = Path(str(rel_nb_path)[:-len(f".{file_ext}")])
            return {
                "name": name,
                "cl_name": Path(*name.parts[1:]),
                "basename": Path(name).stem,
                "format": loc.format,
                "file_ext": file_ext,
                "cl_path": loc.path,
            }

In [ ]:
root_path = '../../test_proj'
root_path, config = get_project_root_and_config(root_path)
get_nb_path_info('../../test_proj/nbs/notebook1.ipynb', '../../test_proj', config)

{'name': Path('nbs/notebook1'),
 'cl_name': Path('notebook1'),
 'basename': 'notebook1',
 'format': 'ipynb',
 'file_ext': 'ipynb',
 'cl_path': 'nbs'}

In [ ]:
show_doc(nblite.utils.get_nb_module_export_name)

---

### get_nb_module_export_name

>      get_nb_module_export_name (nb_path:str, lib_path:str)

In [ ]:
#|export
def get_nb_module_export_name(nb_path: str, lib_path: str) -> str:
    import nbdev.export
    exp = nbdev.export.ExportModuleProc()
    nb = nbdev.export.NBProcessor(nb_path, [exp], debug=False)
    nb.process()
    for mod,cells in exp.modules.items():
        if nbdev.export.first(1 for o in cells if o.cell_type=='code'):
            all_cells = exp.in_all[mod]
            nm = getattr(exp, 'default_exp', None) if mod=='#' else mod
            return nm
    return None
    return (Path(lib_path)/(nm + ".py")).resolve().as_posix()

In [ ]:
get_nb_module_export_name('../../test_proj/nbs/submodule/notebook3.ipynb', '../../test_proj/my_module')

'submodule.notebook3'

In [ ]:
show_doc(nblite.utils.get_nb_module_export_path)

---

### get_nb_module_export_path

>      get_nb_module_export_path (nb_path:str, lib_path:str)

In [ ]:
#|export
def get_nb_module_export_path(nb_path: str, lib_path: str) -> str:
    nb_mod_export_name = get_nb_module_export_name(nb_path, lib_path)
    return (Path(lib_path) / nb_mod_export_name.replace('.', '/')).with_suffix('.py').resolve()

In [ ]:
get_nb_module_export_path('../../test_proj/nbs/submodule/notebook3.ipynb', '../../test_proj/my_module')

Path('/Users/lukastk/dev-proj/nblite/test_proj/my_module/submodule/notebook3.py')

In [ ]:
show_doc(nblite.utils.is_code_loc_nb)

---

### is_code_loc_nb

>      is_code_loc_nb (nb_path:str, root_path:str,
>                      config:nblite.config.NBLiteConfig)

*Returns True if the notebook is a notebook associated with a code location.*

In [ ]:
#|export
def is_code_loc_nb(nb_path: str, root_path: str, config: NBLiteConfig):
    """Returns True if the notebook is a notebook associated with a code location."""
    nb_path = Path(nb_path).resolve()
    root_path = Path(root_path).resolve()
    try:
        rel_nb_path = nb_path.relative_to(root_path)
        for loc in config.code_locations.values():
            if str(rel_nb_path).startswith(loc.path):
                in_cl = True
                is_nb = str(rel_nb_path).endswith(loc.file_ext)
                return in_cl and is_nb
        return False
    except ValueError:
        return False

In [ ]:
root_path = '../../test_proj'
print(is_code_loc_nb('../../test_proj/nbs/notebook1.ipynb', root_path, config))
print(is_code_loc_nb('../../test_proj/nbs/notebook1.pct.py', root_path, config))
print(is_code_loc_nb('../../test_proj/test.txt', root_path, config))

True
False
False


In [ ]:
show_doc(nblite.utils.get_nb_twin_paths)

---

### get_nb_twin_paths

>      get_nb_twin_paths (nb_path:str, root_path:str)

*For a given notebook in a code location, returns the paths to all its 'twins' (the corresponding notebooks in the other code locations).
The original given notebook path is also returned.*

In [ ]:
#|export
def get_nb_twin_paths(nb_path: str, root_path: str):
    """For a given notebook in a code location, returns the paths to all its 'twins' (the corresponding notebooks in the other code locations).
    The original given notebook path is also returned."""
    if root_path is None:
        root_path, config = get_project_root_and_config()
    else:
        root_path = Path(root_path)
        config = read_config(root_path / nblite_config_file_name)
    nb_path = Path(nb_path).resolve()
    cl_nb_name = get_nb_path_info(nb_path, root_path, config)['cl_name']
    
    nb_twins = []
    for loc in config.code_locations.values():
        if loc.format == 'module':
            twin_path = Path(get_nb_module_export_path(nb_path, root_path / loc.path))
        else:
            twin_path = root_path / loc.path / cl_nb_name.with_suffix('.' + loc.file_ext)
        nb_twins.append(twin_path)
        
    nb_twins = tuple(sorted([fp.as_posix() for fp in nb_twins]))
    return nb_twins

In [ ]:
get_nb_twin_paths('../../test_proj/nbs/folder/notebook4.ipynb', '../../test_proj')

('../../test_proj/lgts/folder/notebook4.lgt.py',
 '../../test_proj/nbs/folder/notebook4.ipynb',
 '../../test_proj/pcts/folder/notebook4.pct.py',
 '/Users/lukastk/dev-proj/nblite/test_proj/my_module/notebook4.py')

In [ ]:
show_doc(nblite.utils.get_code_location_nbs)

---

### get_code_location_nbs

>      get_code_location_nbs (root_path:str, cl:nblite.config.CodeLocation,
>                             ignore_underscores:bool=True)

*Returns all notebooks in a code location. If ignore_underscores is True,
notebooks with underscores in their names, or notebooks in folders that start with underscores, are ignored.*

In [ ]:
#|export
def get_code_location_nbs(root_path: str, cl: CodeLocation, ignore_underscores: bool = True):
    """Returns all notebooks in a code location. If ignore_underscores is True,
    notebooks with underscores in their names, or notebooks in folders that start with underscores, are ignored."""
    
    cl_path = Path(root_path) / cl.path
    if not cl_path.exists(): raise ValueError(f"Code location path '{cl_path}' does not exist.")
    
    nbs = []
    for fp in cl_path.glob('**/*'):
        rel_fp = fp.relative_to(cl_path)
        if fp.is_file() and fp.name.endswith(cl.file_ext):
            if ignore_underscores and any(p.startswith('_') for p in rel_fp.parts): continue
            nbs.append(fp)
    return nbs

In [ ]:
get_code_location_nbs('../../test_proj', CodeLocation(path='nbs', format='ipynb'))

[Path('../../test_proj/nbs/notebook2.ipynb'),
 Path('../../test_proj/nbs/notebook1.ipynb'),
 Path('../../test_proj/nbs/submodule/notebook3.ipynb'),
 Path('../../test_proj/nbs/folder/notebook4.ipynb')]

In [ ]:
get_code_location_nbs('../../test_proj', CodeLocation(path='nbs', format='ipynb'), ignore_underscores=False)

[Path('../../test_proj/nbs/notebook2.ipynb'),
 Path('../../test_proj/nbs/notebook1.ipynb'),
 Path('../../test_proj/nbs/_notebook6.ipynb'),
 Path('../../test_proj/nbs/submodule/notebook3.ipynb'),
 Path('../../test_proj/nbs/folder/notebook4.ipynb'),
 Path('../../test_proj/nbs/_scratch/notebook5.ipynb')]

In [ ]:
show_doc(nblite.utils.is_nb_unclean)

---

### is_nb_unclean

>      is_nb_unclean (nb_path:Optional[str]=None,
>                     file_content:Optional[str]=None)

In [ ]:
#|export
def is_nb_unclean(nb_path:Union[str, None]=None, file_content:Union[str, None]=None):
    import nbformat
    from nbconvert.preprocessors import ExecutePreprocessor

    if nb_path is not None and file_content is not None:
        raise ValueError("Only one of nb_path or file_content can be provided.")
    
    if nb_path is None and file_content is None:
        raise ValueError("Either nb_path or file_content must be provided.")

    if nb_path:
        nb_path = Path(nb_path)
        if not nb_path.as_posix().endswith('.ipynb'):
            raise ValueError(f"Error: '{nb_path}' is not a Jupyter notebook file.")

        with open(nb_path) as f:
            nb = nbformat.read(f, as_version=4)
    else:
        nb = nbformat.reads(file_content, as_version=4)

    if nb.metadata: return True

    for cell in nb.cells:
        if cell['cell_type'] != 'code': continue
        if cell['execution_count'] is not None: return True
        if cell.metadata: return True
        for output in cell.get('outputs', []):
            if 'execution_count' in output and output['execution_count'] is not None: return True
            if 'metadata' in output and output['metadata']: return True

    return False

In [ ]:
is_nb_unclean(file_content='{"cells":[]}')

False

In [ ]:
is_nb_unclean('../../test_proj/nbs/notebook1.ipynb')

True

In [ ]:
show_doc(nblite.utils.get_unclean_nbs)

---

### get_unclean_nbs

>      get_unclean_nbs (root_path:str=None, ignore_underscores:bool=False)

*Get all notebooks that have metadata or execution count.

Returns:
    bool: True if all notebooks are clean, False otherwise.*

In [ ]:
#|export
def get_unclean_nbs(root_path: str = None, ignore_underscores: bool = False):
    """
    Get all notebooks that have metadata or execution count.
    
    Returns:
        bool: True if all notebooks are clean, False otherwise.
    """
    if root_path is None:
        root_path, config = get_project_root_and_config()
    else:
        root_path = Path(root_path)
        config = read_config(root_path / nblite_config_file_name)
    
    unclean_nbs = []
    for cl in config.code_locations.values():
        if not cl.format == 'ipynb': continue
        cl_nbs = get_code_location_nbs(root_path, cl, ignore_underscores=ignore_underscores)
        unclean_nbs.extend([nb_path.relative_to(root_path) for nb_path in cl_nbs if is_nb_unclean(nb_path)])
    return unclean_nbs

In [ ]:
get_unclean_nbs('../../test_proj')

[Path('nbs/notebook1.ipynb')]

In [ ]:
show_doc(nblite.utils.get_relative_path)

---

### get_relative_path

>      get_relative_path (from_path:str, to_path:str)

*Returns the relative path to the root path.*

In [ ]:
#|export
def get_relative_path(from_path: str, to_path: str):
    """Returns the relative path to the root path."""
    return Path(os.path.relpath(Path(to_path).resolve(), start=Path(from_path).resolve()))

In [ ]:
get_relative_path('.', '/Users/lukastk/')

Path('../../../..')